In [1]:
import os
from envs.race_strategy_event import RaceEnv
from race_simulation.racesim.src.import_pars import import_pars
from race_simulation.main_racesim import main
from tqdm import tqdm
from collections import defaultdict
import numpy as np
import pandas as pd


In [2]:
directory = "race_simulation/racesim/input/parameters"
MCS_PARS_FILE = 'pars_mcs.ini'

good = []

for entry in tqdm(os.scandir(directory)):
    if entry.is_file:
        filename = entry.name
        if filename != "pars_mcs.ini":
            pars_in, vse_paths = import_pars(use_print=False,
                                            use_vse=False,
                                            race_pars_file=filename,
                                            mcs_pars_file=MCS_PARS_FILE)
            if len(pars_in['event_pars']['fcy_data']['phases']) == 0:
                good.append(filename)

41it [00:00, 129.43it/s]WARNING: Parameter t_gap_overtake_vel is None, continuing with 0.0s!
122it [00:00, 133.78it/s]


In [3]:
for race in tqdm(good):
    for i in range(100):
        # ------------------------------------------------------------------------------------------------------------------
        # USER INPUT -------------------------------------------------------------------------------------------------------
        # ------------------------------------------------------------------------------------------------------------------

        # set race parameter file names
        race_pars_file_ = race
        mcs_pars_file_ = 'pars_mcs.ini'

        # set simulation options
        # use_prob_infl:        activates probabilistic influences within the race simulation -> lap times, pit stop
        #                       durations, race start performance
        # create_rand_events:   activates the random creation of FCY (full course yellow) phases and retirements in the race
        #                       simulation -> they will only be created if the according entries in the parameter file
        #                       contain empty lists, otherwise the file entries are used
        # use_vse:              determines if the VSE (virtual strategy engineer) is used to take tire change decisions
        #                       -> the VSE type is defined in the parameter file (VSE_PARS)
        # no_sim_runs:          number of (valid) races to simulate
        # no_workers:           defines number of workers for multiprocess calculations, 1 for single process, >1 for
        #                       multi-process (you can use print(multiprocessing.cpu_count()) to determine the max. number)
        # use_print:            set if prints to console should be used or not (does not suppress hints/warnings)
        # use_print_result:     set if result should be printed to console or not
        # use_plot:             set if plotting should be used or not
        sim_opts_ = {"use_prob_infl": True,
                        "create_rand_events": False,
                        "use_vse": False,
                        "no_sim_runs": 1,
                        "no_workers": 1,
                        "use_print": False,
                        "use_print_result": False,
                        "use_plot": False}

        # ------------------------------------------------------------------------------------------------------------------
        # SIMULATION CALL --------------------------------------------------------------------------------------------------
        # ------------------------------------------------------------------------------------------------------------------
        main(sim_opts=sim_opts_, race_pars_file=race_pars_file_, mcs_pars_file=mcs_pars_file_)

uing with 0.0s!
100%|██████████| 38/38 [13:12<00:00, 20.86s/it]


In [4]:
LOGS_FOLDER = "race_simulation/racesim/output"
DATA_FOLDER = "results"
dfs = defaultdict(dict)

for entry in tqdm(os.scandir(LOGS_FOLDER)):
    if os.path.isdir(entry):
        race_name = entry.name
        directory = os.path.join(LOGS_FOLDER, race_name, DATA_FOLDER)
        for entry in os.scandir(directory):
            if entry.is_file:
                filename = entry.name
                filename = filename[:-4]
                tokens = filename.split('_')
                timestamp = tokens.pop(0)
                df = pd.read_csv(entry.path)
                dfs[tokens[0] + "_" + timestamp][tokens[2]] = df
                dfs[tokens[0] + "_" + timestamp]['year'] = tokens[1]

38it [01:15,  1.99s/it]


In [5]:
# database = pd.DataFrame(columns=['race', 'year', 'lap', 'driver', 'lap_time', 'cumulative_time', 'influence', 'position', 'id'])
database = pd.DataFrame(columns=['race', 'year', 'lap', 'driver', 'cumulative_time', 'position', 'id'])
index = 0
count = 0
laps = [database]
for i, race_id in enumerate(tqdm(dfs)):
    tokens = race_id.split('_')
    timestamp = tokens[1]
    race = tokens[0]
    year = dfs[race_id]['year']

    # lap_times = dfs[race_id]['laptimes']
    # influences = dfs[race_id]['lapinfluences']
    positions = dfs[race_id]['positions']
    race_times = dfs[race_id]['racetimes']

    final_lap = max(race_times['lap'].unique())
    
    columns = list(race_times.columns)
    columns.remove('lap')

    #for lap in lap_times.lap.unique():
        # lap_time_single_lap = lap_times.loc[lap_times['lap'] == lap]
        # influences_single_lap = influences.loc[influences['lap'] == lap]
    positions_single_lap = positions.loc[positions['lap'] == final_lap]
    for col in columns:
        max_race_time = race_times[col].max()
        lap = race_times.loc[race_times[col] == max_race_time, 'lap']
        if len(lap.values)> 0:
            lap = lap.values[0]
        else:
            lap = 1
        index += 1
        new_df = {}
        new_df['id'] = [timestamp]
        new_df['race'] = [race]
        new_df['year'] = [year]
        new_df['lap'] = [lap]
        new_df['driver'] = [col]
        # new_df['lap_time'] = lap_time_single_lap[col].values
        # new_df['influence'] = influences_single_lap[col].values
        new_df['cumulative_time'] = [max_race_time]
        new_df['position'] = positions_single_lap[col].values
        new_df = pd.DataFrame(new_df, index=[index])
        laps.append(new_df)
database = pd.concat(laps)


100%|██████████| 3800/3800 [01:36<00:00, 39.36it/s]


In [6]:
database

,race,year,lap,driver,cumulative_time,position,id
1,Austin,2017,55,VAN,5739.079,14,1607608799505
2,Austin,2017,14,RIC,1464.368,18,1607608799505
3,Austin,2017,56,VET,5710.212,2,1607608799505
4,Austin,2017,56,RAI,5720.440,4,1607608799505
5,Austin,2017,54,GRO,5684.995,15,1607608799505
...,...,...,...,...,...,...,...
78396,YasMarina,2019,55,SAI,5801.822,9,1607609591947
78397,YasMarina,2019,54,RUS,5768.087,16,1607609591947
78398,YasMarina,2019,55,BOT,5785.667,5,1607609591947
78399,YasMarina,2019,54,KUB,5775.022,19,1607609591947


In [14]:
dataframes = []

for race in tqdm(database['race'].unique()):
    race_final = database[database['race'] == race]
    for year in race_final['year'].unique():
        race_year_final = race_final.loc[race_final['year'] == year]
        max_lap = race_year_final['lap'].max()

        # Check if VET has retired
        max_vet = race_year_final.loc[race_year_final['driver'] == 'VET', "lap"].max()
        if max_vet < max_lap - 2:
            print("VET retired or was lapped at", race, year)
        else:
            to_leader = []
            to_front = []
            to_following = []
            classification = []
           
            for ident in race_year_final['id'].unique():
                final_lap = race_year_final.loc[(race_year_final['id'] == ident) & (race_year_final['driver'] == 'VET'), "lap"].values[0]
                if final_lap == max_lap:
                    # Take VET' scumulative time
                    vet_time = race_year_final.loc[(race_year_final['id'] == ident) & (race_year_final['driver'] == 'VET'), 'cumulative_time'].values[0]
                    # Take VET's final position
                    position = race_year_final.loc[(race_year_final['id'] == ident) & (race_year_final['driver'] == 'VET'), 'position'].values[0]
                    classification.append(position)
                    if position != 1:
                        # Compute winner's cumulative time
                        leader_time = race_year_final.loc[(race_year_final['id'] == ident) & (race_year_final['position'] == 1), 'cumulative_time'].values[0]
                        # Compute driver in front's cumulative time
                        if position > 2:
                            in_front_time = race_year_final.loc[(race_year_final['id'] == ident) & (race_year_final['position'] == position - 1), 'cumulative_time'].values[0]
                        if position == 2:
                            in_front_time = leader_time
                    else:
                        in_front_time = leader_time = vet_time

                    if vet_time - in_front_time < 0:
                        print(max_vet, max_lap, position - 1)
                        print(race_year_final.loc[(race_year_final['id'] == ident)])

                    to_front.append(vet_time - in_front_time)
                    to_leader.append(vet_time - leader_time)

                    # Compute following driver's cumulative time
                    following_time = race_year_final.loc[(race_year_final['id'] == ident) & (race_year_final['position'] == position + 1)]
                    if len(following_time) > 0:
                        following_time = following_time['cumulative_time'].values[0]
                    else:
                        following_time = vet_time

                    to_following.append(following_time - vet_time)

            size = len(to_leader)
            dataframes.append(pd.DataFrame({"race": [race] * size, "year" : [year] * size, "position": classification, "to_leader": to_leader, "to_front": to_front, "to_following": to_following}))
            data =  pd.concat(dataframes)

            
                



                 

100%|██████████| 17/17 [00:09<00:00,  1.82it/s]


In [15]:
for year in sorted(data['year'].unique()):
    year_data = data[data['year'] == year]
    for race in sorted(year_data['race'].unique()):
        race_year_data = year_data[year_data['race'] == race]
        print(race, year)
        print(race_year_data.describe())
        print()

KualaLumpur 2014
         position   to_leader    to_front  to_following
count  100.000000  100.000000  100.000000    100.000000
mean     1.070000    0.577010    0.577010     15.399680
std      0.256432    2.487242    2.487242      9.814131
min      1.000000    0.000000    0.000000      0.400000
25%      1.000000    0.000000    0.000000      9.315000
50%      1.000000    0.000000    0.000000     13.212500
75%      1.000000    0.000000    0.000000     20.230000
max      2.000000   16.474000   16.474000     57.790000

Monza 2014
        position  to_leader   to_front  to_following
count  31.000000  31.000000  31.000000     31.000000
mean    6.870968  81.656194  11.043742    -21.598742
std     1.627717   8.376731  11.706637     38.613983
min     5.000000  64.656000   0.400000    -87.213000
25%     6.000000  79.467000   2.109500    -73.177500
50%     7.000000  84.039000   6.337000      0.400000
75%     7.500000  88.257000  17.776000      2.861500
max    10.000000  91.772000  36.671000     